<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Data-Augmentation/Analysis" data-toc-modified-id="Data-Augmentation/Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Augmentation/Analysis</a></span></li><li><span><a href="#Dataloader-creation-and-test" data-toc-modified-id="Dataloader-creation-and-test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataloader creation and test</a></span></li><li><span><a href="#Model-creation" data-toc-modified-id="Model-creation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model creation</a></span></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model Training</a></span><ul class="toc-item"><li><span><a href="#Model-Testing" data-toc-modified-id="Model-Testing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Model Testing</a></span></li></ul></li></ul></div>

**GET DATA**

Summary of this notebook: ...

Definition of Done: ...

# Imports


In [1]:
import os
from os import path
import sys
import math
import cv2 # Read raw image
import glob
import random
import numpy as np
import pickle
from matplotlib import pyplot as plt
from scipy import ndimage # For rotation task or
import imutils
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
from skimage.transform import resize
from pprint import pprint
import tqdm

import tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, AveragePooling2D, MaxPooling2D, Reshape, Conv2DTranspose, ZeroPadding2D, Add
from tensorflow.keras.layers import Activation, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import PReLU

sys.path.append("/home/satyarth934/code/FDL_2020/training_scripts")
import utils

# Check to see if GPU is being used
print(tensorflow.test.gpu_device_name())
print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

/device:GPU:0
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Num GPUs Available:  2


In [2]:
DATA_PATH = "/home/satyarth934/data/modis_data_products/*/array_3bands_normalized/448/*"

img_paths = glob.glob(DATA_PATH)
train_test_split = 0.8
X_test_paths = img_paths[int(train_test_split * len(img_paths)):]
print(len(X_test_paths))

dims = (448, 448, 3)

# Loading Data
X_test = utils.getData(X_test_paths, dims)
X_test = utils.normalize(X_test)

146


In [3]:
# nan_val = utils.nansInData(X_test)
# plt.scatter(np.arange(1,len(nan_val)+1), nan_val)

# print("Usable:", len([v for v in nan_val if v<20000]))

# print("Mahashay:", [(i, v) for i, v in enumerate(nan_val) if v>500000])

In [ ]:
# 45 pixels: radius of the locality
# plt.imshow(X_test[10])
# plt.show()

import random
from multiprocessing import Pool


def getClosest(pt, options):
    x,y = pt
    min_val = -9999
    min_pt = options[0]
    for o in options:
        sse = np.sum(np.square(np.array(pt) - np.array(o)))
        if sse < min_val:
            min_val = se
            min_pt = o
    
    return min_pt


N = 2
radius = 100

# local region
(x,y,z) = np.where(np.isnan(X_test[10]))

# nan value indices
nan_idxs = [_ for _ in zip(x,y)]
nan_idxs = list(set(nan_idxs))

attempts = 0

clone = np.copy(X_test[10])

for i, item in enumerate(tqdm.tqdm_notebook(nan_idxs)):
    x,y = item
    # region
    xtl, xtr = max(0, x - radius), min(X_test[10].shape[1], x + radius)
    ytl, ytr = max(0, y - radius), min(X_test[10].shape[1], y + radius)
    
    random_idxs = []
    counter = 0
    att2 = 0
    flag = 0
    while counter < N:
        if att2 > N * 10:
            flag = 1
            break
        att2 += 1
        
        attempts += 1
        random_idx = np.random.randint(xtl, xtr), np.random.randint(ytl, ytr)
#         random_idx = (1,2)
        if np.sum(np.isnan(X_test[10][random_idx[0], random_idx[1],:])) > 0:
            continue
        random_idxs.append(random_idx)
        counter += 1
    
    if i%1000 == 0:
        print("attempts:", attempts / ((i+1) * N))
    
    if(flag==1):
        continue

#     closest_x, closest_y = random_idx
#     print("===>>", *zip(random_idx[0], random_idx[1]))
    closest_x, closest_y = getClosest((x,y), random_idxs)
#     print(x_rand)
#     print(y_rand)
#     print(closest_x, closest_y)
#     print("------")
    clone[x,y,:] = X_test[10][closest_x,closest_y,:]

plt.imshow(clone)
# plt.savefig("n%drad%dA3.png" % (N, radius))
plt.show()

In [ ]:
img1 = plt.imread('n3rad45.png')
img2 = plt.imread('n1rad45.png')

print(np.sum(img1 - img2))